# Mise en commun du code et du parser pour jeux de mots

In [25]:
import time
import requests
import shutil
from os import remove, mkdir, rmdir
from os.path import exists
import pandas as pd 
from os.path import exists
from alive_progress import alive_bar

# Arborescence du sysème de fichier 
PATH            = "data/"
PATH_REQUEST    = "data/requests/"
PATH_DEF        = "data/def/"
PATH_E          = "data/e/"
PATH_NT         = "data/nt/"
PATH_RE         = "data/re/"
PATH_RS         = "data/rs/"
PATH_RT         = "data/rt/"

# Extension des fichiers 
EXT_REQUEST     = ".txt"
EXT_DEF         = "_def.csv"
EXT_E           = "_e.csv"
EXT_NT          = "_nt.csv"
EXT_RE          = "_re.csv"
EXT_RS          = "_rs.csv"
EXT_RT          = "_rt.csv"

class Terme: 
    def __init__(self, mot):
        self.mot = mot

        if self.isKnow(): 
            self.load()
        else: 
            self.download()

    def getMot(self): 
        return self.mot

    def getID(self): 
        return self.id

    def isKnow(self):
        return exists(PATH_REQUEST + self.mot + EXT_REQUEST)

    def request(self): 
        url = 'https://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=' + self.mot + '&rel='
        headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
        r = requests.post(url, headers=headers)

        # Enregistrer la requetes dans un fichier text
        f = open(PATH_REQUEST + self.mot + EXT_REQUEST, "w")
        f.write(r.text)
        f.close()

        f = open(PATH_REQUEST + self.mot + EXT_REQUEST, "r")
        lines = f.readlines()
        f.close()

        requestIsGood = "<br>//&nbsp; &nbsp; &nbsp; WARNING TROP GROS.<br>TOWARD CACHE<br>\n" not in lines and "<CODE>MUTED_PLEASE_RESEND\n" not in lines and "<div class=\"jdm-warning\"><br>Le terme '" + self.mot + "' n'existe pas !</div></div>\n"

        if not requestIsGood: 
            remove(PATH_REQUEST + self.mot + EXT_REQUEST)
            

        return requestIsGood

    def download(self):
        i = 0

        while not self.request() and i < 5: 
            i += 1

        self.load()

    def load(self):
        f = open(PATH_REQUEST + self.mot + EXT_REQUEST, "r")
        lines = f.readlines()
        try : 
            code = [lines.index('<CODE>\n'), lines.index('</CODE>\n') +1]
        except ValueError: 
            code = [0, 0]

        index = {}

        for i in range(code[0], code[1]):
            if lines[i] == "<def>\n":
                index["def"] = i

            if lines[i] == "// les types de noeuds (Nodes Types) : nt;ntid;'ntname'\n":
                index["nt"] = i

            if lines[i] == "// les noeuds/termes (Entries) : e;eid;'name';type;w;'formated name' \n":
                index["e"] = i

            if lines[i] == "// les types de relations (Relation Types) : rt;rtid;'trname';'trgpname';'rthelp' \n":
                index["rt"] = i

            if lines[i] == "// les relations sortantes : r;rid;node1;node2;type;w \n":
                index["rs"] = i

            if lines[i] == "// les relations entrantes : r;rid;node1;node2;type;w \n":
                index["re"] = i 

            if lines[i] == "// END\n":
                index["end"] = i 
    

        #index
        m_def   = open(PATH_DEF + self.mot + EXT_DEF, "w")
        m_nt    = open(PATH_NT  + self.mot + EXT_NT,  "w")
        m_e     = open(PATH_E   + self.mot + EXT_E,   "w")
        m_rt    = open(PATH_RT  + self.mot + EXT_RT,  "w")
        m_rs    = open(PATH_RS  + self.mot + EXT_RS,  "w")
        m_re    = open(PATH_RE  + self.mot + EXT_RE,  "w")

        # Head
        m_nt.write("nt;ntid;ntname\n")
        m_e.write("e;eid;name;type;w;help\n")
        m_rt.write("rt;rtid;trname;trgpname;rthelp\n")
        m_rs.write("r;rid;node1;node2;type;w\n")
        m_re.write("r;rid;node1;node2;type;w\n")

        try: 
            for i in range(index['def'], index['nt']):
                m_def.write(lines[i])

            for i in range(index['nt']+2, index['e']):
                m_nt.write(lines[i].replace("'", ""))

            for i in range(index['e']+2, index['rt']):
                if lines[i].count(";") <= 5:
                    m_e.write(lines[i].replace("'", ""))

            for i in range(index['rt']+2, index['rs']):
                m_rt.write(lines[i].replace(" ; ", " ").replace("'", "")) # Modifier les séparateur pour éviter les pb à l'ouverture

            if 're' in index and 'rs' in index: 
                for i in range(index['rs']+2, index['re']):
                    m_rs.write(lines[i])

                for i in range(index['re']+2, index['end']):
                    m_re.write(lines[i])
            elif 'rs' in index: 
                for i in range(index['rs']+2, index['end']):
                    m_rs.write(lines[i])
            elif 're' in index: 
                for i in range(index['re']+2, index['end']):
                    m_re.write(lines[i])
        except KeyError: 
            pass

        m_def.close()
        m_nt.close()
        m_e.close()
        m_rt.close()
        m_rs.close()
        m_re.close()

        self.E  = pd.read_csv(PATH_E  + self.mot + EXT_E,  sep=";")
        self.NT = pd.read_csv(PATH_NT + self.mot + EXT_NT, sep=";")
        self.RE = pd.read_csv(PATH_RE + self.mot + EXT_RE, sep=";")
        self.RS = pd.read_csv(PATH_RS + self.mot + EXT_RS, sep=";")
        self.RT = pd.read_csv(PATH_RT + self.mot + EXT_RT, sep=";")
        self.R  = pd.concat([self.RS, self.RE])

        if self.RS.index > 0:
            self.id = int(self.RS['node1'][0])
        else
            self.id = -1

        #self.id = int(self.E.loc[self.E['name'] == self.mot]['eid'][0])

    def initMyLocalDataBase(): 
        if not exists(PATH): 
            mkdir(PATH)

        if not exists(PATH_DEF): 
            mkdir(PATH_DEF)

        if not exists(PATH_REQUEST): 
            mkdir(PATH_REQUEST)

        if not exists(PATH_E): 
            mkdir(PATH_E)

        if not exists(PATH_NT): 
            mkdir(PATH_NT)

        if not exists(PATH_RE): 
            mkdir(PATH_RE)

        if not exists(PATH_RS): 
            mkdir(PATH_RS)

        if not exists(PATH_RT): 
            mkdir(PATH_RT)

    def deleteMyLocalDataBase():
        if exists(PATH):
            shutil.rmtree(PATH)

    def isValidTerme(terme): 
        return " " not in terme and "<" not in terme and">" not in terme and":" not in terme and "\x9c" not in terme and "ï" not in terme and "_" not in terme

## Construction de l'arborescence 

In [23]:
Terme.initMyLocalDataBase() 

## Les fonctions pour extraire les relations

In [26]:
A = Terme("toto")
B = Terme("Suisse")
C = Terme("skate")
D = Terme("bite")
E = Terme("drogue")
F = Terme("sale")
G = Terme("gueez")
H = Terme("bonheur")
I = Terme("objet")

IndexError: index 0 is out of bounds for axis 0 with size 0

In [4]:
def relationDistance0(A):
    relation = A.RS.loc[A.RS['node2'] == A.getID()]

    relation_A_B = {}

    with alive_bar(len(relation.index), force_tty=True) as bar:
        for i, row_i in relation.iterrows() : 
            relation_A_B[A.getMot() + " " + A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False) + " " + A.getMot()] = row_i['w']
            bar() 

    relation_A_B_sorted = {}

    for i in sorted(relation_A_B, key=relation_A_B.get, reverse=True):
        relation_A_B_sorted[i] = relation_A_B[i]

    for r in relation_A_B_sorted: 
        print(str(r) + " : " + str(relation_A_B_sorted[r]))



In [5]:
relationDistance0(A)

|████████████████████████████████████████| 1/1 [100%] in 0.0s (25.36/s)                                                 
toto r_lemma toto : 55


In [6]:
def relationDistance1(A, B): 
    relation = A.RS.loc[A.RS['node2'] == B.getID()]

    relation_A_B = {}

    with alive_bar(len(relation.index), force_tty=True) as bar:
        for i, row_i in relation.iterrows() : 
            relation_A_B[A.getMot() + " " + A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False) + " " + B.getMot()] = row_i['w']
            bar()

    relation_A_B_sorted = {}

    for i in sorted(relation_A_B, key=relation_A_B.get, reverse=True):
        relation_A_B_sorted[i] = relation_A_B[i]

    for r in relation_A_B_sorted: 
        print(str(r) + " : " + str(relation_A_B_sorted[r]))
    
     

In [35]:
relationDistance1(A, B)

|████████████████████████████████████████| 2/2 [100%] in 0.0s (54.02/s)                                                 
toto glose/sens/signification Suisse : 100
toto idée associée Suisse : 15


In [7]:
def relationDistance2(A, B):
    c = 0
    termes = []

    for eidS in A.RS['node2']: 
        tmp = len(B.RE.loc[B.RE['node1'] == eidS])
        
        if tmp > 0: 
            terme = A.E.loc[A.E['eid'] == eidS]['name'].to_string(index=False)
            if Terme.isValidTerme(terme):
                termes.append(terme)

        c += tmp 

    termes = list(dict.fromkeys(termes))
    
    print(termes)
    print(c)

    relation_A_B = {}

    with alive_bar(len(termes) + 1, force_tty=True) as bar:
        for terme in termes:     
            AB = Terme(terme)

            relation_A_AB = A.RS.loc[A.RS['node2'] == AB.getID()]
            bar()
            for i, row_i in relation_A_AB.iterrows(): 
                relation_AB_B = B.RE.loc[B.RE['node1'] == AB.getID()]

                for j, row_j in relation_AB_B.iterrows():
                    relation_A_B[A.getMot() + " " + A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False) + " " + AB.getMot() + " " + B.RT.loc[B.RT['rtid'] == row_j['type']]['trgpname'].to_string(index=False) + " " + B.getMot()] = row_i['w'] + row_j['w'] / 2
                    
        bar()

    relation_A_B_sorted = {}

    for i in sorted(relation_A_B, key=relation_A_B.get, reverse=True):
        relation_A_B_sorted[i] = relation_A_B[i]

    for r in relation_A_B_sorted: 
        print(str(r) + " : " + str(relation_A_B_sorted[r]))

In [8]:
relationDistance2(A, B)

['linguistique', 'Suisse', 'enfant', 'tête', 'langue', 'toto', 'jambe', 'personne', 'visage', 'jambes', 'nez', 'bouche', 'politique', 'main', 'cou', 'pied', 'coeur', 'bras', 'mourir', 'vivre', 'dormir', 'parler', 'individu', 'adn', 'yeux', 'ADN', 'corps', 'oeil', 'vivant', 'regarder', 'mort', 'maladie', 'école', 'eye', 'bonheur', 'histoire', 'addition', 'informatique']
117
|████████████████████████████████████████| 39/39 [100%] in 16.7s (2.34/s)                                               
toto partie ADN chose>lieu Suisse : 228.5
toto domaine linguistique r_wiki Suisse : 190.0
toto idée associée ADN chose>lieu Suisse : 181.5
toto glose/sens/signification Suisse r_lemma Suisse : 130.0
toto glose/sens/signification Suisse idée associée Suisse : 112.5
toto glose/sens/signification Suisse chose>lieu Suisse : 112.5
toto glose/sens/signification Suisse lieu>chose Suisse : 112.5
toto r_lemma toto glose/sens/signification Suisse : 105.0
toto idée associée enfant chose>lieu Suisse : 94.0
tot

In [9]:
def relationDistance2_RS(A, B):
    termes = []

    for eidS in A.RS['node2']: 
        terme = A.E.loc[A.E['eid'] == eidS]['name'].to_string(index=False)

        if Terme.isValidTerme(terme):
            termes.append(terme)


    termes = list(dict.fromkeys(termes))
    
    print(termes)
    print(len(termes))

    relation_A_B = {}

    with alive_bar(len(termes) + 1, force_tty=True) as bar:
        for terme in termes:     
            AB = Terme(terme)
            relation_A_AB = A.RS.loc[A.RS['node2'] == AB.getID()]
            bar()

            for i, row_i in relation_A_AB.iterrows(): 
                relation_AB_B = AB.RS.loc[AB.RS['node2'] == B.getID()]

                for j, row_j in relation_AB_B.iterrows():
                    relation_A_B[A.getMot() + " " + A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False) + " " + AB.getMot() + " " + B.RT.loc[B.RT['rtid'] == row_j['type']]['trgpname'].to_string(index=False) + " " + B.getMot()] = row_i['w'] + row_j['w'] / 2
                    
        bar()

    relation_A_B_sorted = {}

    for i in sorted(relation_A_B, key=relation_A_B.get, reverse=True):
        relation_A_B_sorted[i] = relation_A_B[i]

    for r in relation_A_B_sorted: 
        print(str(r) + " : " + str(relation_A_B_sorted[r]))

In [10]:
relationDistance2_RS(A, H)

['pou', 'linguistique', 'blague', 'entomologie', 'zéro', 'Suisse', 'enfant', 'tête', 'langue', 'type', 'Toto', 'patte', 'blagues', 'coelomate', 'toto', 'coelomé', 'humour', 'animal', 'insecte', 'célomate', 'prénom', 'nom', 'eucaryote', 'opisthoconte', 'uniconte', 'filozoaire', 'holozoaire', 'choano-organisme', 'choanobionte', 'métazoaire', 'eumétazoaire', 'invertébré', 'bilatérien', 'hyponeurien', 'protostomien', 'ecdysozoaire', 'panarthropode', 'euarthropode', 'arthropode', 'mandibulate', 'mandibulé', 'pancrustacé', 'hexapode', 'jambe', 'cancre', 'personne', 'visage', 'petit', 'toto-résistance', 'jambes', 'nez', 'bouche', 'politique', 'entreprises', 'main', 'cou', 'botanique', 'pied', 'élève', 'coeur', 'débile', 'personnage', 'bras', 'mourir', 'vivre', 'ti', 'nerf', 'dormir', 'parler', 'manger', 'brun', 'insectes', 'individu', 'entreprise', 'adn', 'zoologie', 'astucieux', 'Totò', 'partisan', 'yeux', 'abdomen', 'faune', 'langage', 'ADN', 'corps', 'oeil', 'pattes', 'sexe', 'thorax', 'an

In [11]:
def relationDistance3_profodeur(A, D): 
    # A -> B -> C -> D
    sortie_A = []

    for eidS in A.RS['node2']: 
        terme_A = A.E.loc[A.E['eid'] == eidS]['name'].to_string(index=False)

        if Terme.isValidTerme(terme=terme_A):
            B = Terme(terme_A)

            for eidS in B.RS['node2']: 
                terme_B = B.E.loc[B.E['eid'] == eidS]['name'].to_string(index=False)

                if Terme.isValidTerme(terme=terme_B):
                    C = Terme(terme_B)
        
                    for eidS in C.RS['node2']: 
                        terme_C = C.E.loc[C.E['eid'] == eidS]['name'].to_string(index=False)

                        if Terme.isValidTerme(terme=terme_C):
                            print(A.getMot() + " -> " + B.getMot() + " -> " + C.getMot() + " -> " + D.getMot(), end="\r")

                            if terme_C == D.getMot(): 
                                relation_A_B = A.RS.loc[A.RS['node2'] == B.getID()]
                                relation_B_C = B.RS.loc[B.RS['node2'] == C.getID()]
                                relation_C_D = C.RS.loc[C.RS['node2'] == D.getID()]

                                

                                for i, row_i in relation_A_B.iterrows(): 
                                    for j, row_j in relation_B_C.iterrows():
                                        for k, row_k in relation_C_D.iterrows(): 
                                            print(A.getMot() + " " + A.RT.loc[A.RT['rtid'] == row_i['type']]['trgpname'].to_string(index=False) + " " + 
                                                            B.getMot() + " " + B.RT.loc[B.RT['rtid'] == row_j['type']]['trgpname'].to_string(index=False) + " " + 
                                                            C.getMot() + " " + C.RT.loc[C.RT['rtid'] == row_k['type']]['trgpname'].to_string(index=False) + " "  + D.getMot() + " : " + str(row_i['w'] + row_j['w'] + row_k['w']/3))

                                print("")


In [12]:
relationDistance3_profodeur(A, B)

KeyboardInterrupt: 

# Alive progress bar

In [14]:
from alive_progress import alive_bar
import time

with alive_bar(1000, force_tty=True, bar='fish') as bar:
    for i in range(1000):
        time.sleep(0.01)
        bar()

|¸.·´¯`·.·´¯`·.¸¸.·´¯`·.¸.·´¯`·.·´¯`·.¸¸.| 1000/1000 [100%] in 12.0s (83.24/s)                                          


In [15]:
for i in range(10):
    with alive_bar(100, force_tty=True, ctrl_c=False, title=f'Download {i}') as bar:
        for i in range(100):
            time.sleep(0.02)
            bar()

Download 0 |████████████████████████████████████████| 100/100 [100%] in 2.4s (42.23/s)                                  
Download 1 |████████████████████████████████████████| 100/100 [100%] in 2.4s (41.23/s)                                  
Download 2 |████████████████████████████████████████| 100/100 [100%] in 2.4s (41.46/s)                                  
Download 3 |████████████████████████████████████████| 100/100 [100%] in 2.4s (41.08/s)                                  
Download 4 |████████████████████████████████████████| 100/100 [100%] in 2.4s (41.06/s)                                  
Download 5 |████████████████████████████████████████| 100/100 [100%] in 2.4s (41.23/s)                                  
Download 6 |████████████████████████████████████████| 100/100 [100%] in 2.4s (41.55/s)                                  
Download 7 |████████████████████████████████████████| 100/100 [100%] in 2.4s (42.54/s)                                  
Download 8 |████████████████████

In [16]:
import time
for x in range (0,5):  
    b = "Loading" + "." * x
    print (b, end="\r")
    time.sleep(1)

In [46]:
mot = Terme("adn")

id = int(mot.RS['node1'][0])

print(id)
print(mot.getID())

16747
16747


# Delete

In [58]:
Terme.deleteMyLocalDataBase() 

Trouver la requête qui à merdé...

In [15]:

# import required module
import os
# assign directory
directory = 'data/requests'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        t = filename.replace(".txt", "")
        print(t)
        try: 
            T = Terme(t)
        except ValueError: 
            print("=====================================================")
        

drogue
eye
astucieux
Entreprise
politique
bêtises
indifférence
bilatérien
phthiraptère
corps
zéro
bonhomme
pancrustacé
vivant
bras
thorax
parasite
squelette
main
protostome
sexe
blagues
ADN
mandibulé
mandibulate
Totò
choano-organisme
jambe
toto
objet
bite
opisthoconte
dormir
mourir
calcul
sale
insectes
linguistique
satisfaction
surnom
Japon
cancre
type
anxiété
Zoologie
cheveux
surprise
entomologie
internet
langage
patte
choanobionte
coelomé
tristesse
botanique
toto-résistance
personne
manger
mort
drôle
uniconte
peur
pédiculidé
célomate
filozoaire
euarthropode
invertébré
blague
métazoaire
Suisse
personnage
bonheur
abdomen
individu
adn
maladie
hyponeurien
vermine
femelle
cou
tête
antiparasitaire
rire
hexapode
eucaryote
oeil
partisan
regarder
pattes
zoologie
amour
hématophage
nerf
protostomien
informatique
phtiriase
parler
humour
animal
ti
école
insecte
eumétazoaire
pied
crainte
rongeur
entreprises
méfiance
holozoaire
nez
élève
néphrozoaire
bêtise
Toto
addition
arthropode
ecdysozoaire
pas